In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd

vacancy_name = str(input())

s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://hh.ru/"

data = []

browser.get(url)
sleep(5)

input_field = browser.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div[3]/div/div/form/div/div[1]/div/input")
input_field.send_keys(vacancy_name)
browser.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div[3]/div/div/form/div/div[2]/button").click()

sleep(5)

while True:
    sleep(5)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    cards = soup.findAll("div", class_ = "vacancy-serp-item")
    
    for card in cards:
        try:
            name = card.find("a", class_ = "bloko-link").text.strip()
        except:
            name = "-"
        try:
            spans = card.findAll("span", class_ = "bloko-header-section-3 bloko-header-section-3_lite")
            salary1 = spans[1].text.strip().split()
            salary = "".join(salary1)
            salary
        except:
            salary = "-"
        try:
            company = card.find("a", class_ = "bloko-link bloko-link_kind-secondary").text.strip()
        except:
            company = "-"
        try:
            link = card.find("a", class_ = "bloko-link").get("href")
        except:
            link = "-"
        data.append([name, salary, company, link])
    print(len(data), "cards collected")
        
    try:
        browser.find_element(By.CSS_SELECTOR, "[data-qa='pager-next']").click()
    except:
        break
        
headers = ["name", "salary", "company", "link"]
df = pd.DataFrame(data, columns = headers)
df.to_csv("./hhru.csv", sep = ";", encoding = 'utf8')